In [1]:
from joblib import dump, load
import pandas as pd
from sklearn.feature_extraction import FeatureHasher
import numpy as np

In [2]:
column_names = ['interger1','interger2','interger3','interger4','interger5','interger6',
                'interger7','interger8','interger9','interger10','interger11','interger12','interger13',
                'categorical1','categorical2','categorical3','categorical4','categorical5','categorical6',
                'categorical7','categorical8','categorical9','categorical10','categorical11','categorical12',
                'categorical13','categorical14','categorical15','categorical16','categorical17','categorical18',
                'categorical19','categorical20','categorical21','categorical22','categorical23','categorical24',
                'categorical25','categorical26']

In [3]:
test = pd.read_csv('/home/bahbbc/workspace/display-ads-challenge/dac/test.txt', sep='\t',
            names=column_names, chunksize=327433, engine='python')

In [4]:
clt = load('regularized-cat-logistic.pkl')
scaler = load('integer-scaler.pkl')

In [5]:
integer_cols = ['interger2', 'interger8', 'interger5','interger11', 'interger9', 'interger7']
cat_vars = ['categorical8', 'categorical14', 'categorical5', 'categorical12', 'categorical11', 
                'categorical10', 'categorical7', 'categorical1', 'categorical15', 'categorical18', 
                'categorical13', 'categorical16']

In [6]:
feature_hasher = FeatureHasher(n_features=1000)

In [8]:
probs = np.array([0,0])
for chunk in test:
    #pre-processing
    X = chunk[integer_cols + cat_vars]
    X.update(X[integer_cols].fillna(0))
    X.update(X[cat_vars].fillna('NULL'))

    X_integer = scaler.transform(X[integer_cols])
    
    X_cat = feature_hasher.transform(X[cat_vars].to_dict('records')).toarray()
    X_test = np.hstack((X_cat, X_integer))
    #predict
    prob = clt.predict_proba(X_test)
    probs = np.vstack((probs, prob))

/home/bahbbc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  raise_on_error=True)


In [9]:
probs[:10]

array([[ 0.        ,  0.        ],
       [ 0.66769261,  0.33230739],
       [ 0.86358344,  0.13641656],
       [ 0.71143448,  0.28856552],
       [ 0.60382319,  0.39617681],
       [ 0.65524312,  0.34475688],
       [ 0.65183221,  0.34816779],
       [ 0.6445354 ,  0.3554646 ],
       [ 0.72149176,  0.27850824],
       [ 0.71432725,  0.28567275]])

In [10]:
len(probs)

5714703